In [1]:
# 필요한 모듈 import
import fn_files as fn

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import copy
import json
import math

In [2]:
data =pd.read_excel('../01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터.xlsx',header=1)

In [3]:
data.dtypes

방송일시     datetime64[ns]
노출(분)           float64
마더코드              int64
상품코드              int64
상품명              object
상품군              object
판매단가              int64
취급액             float64
dtype: object

### 브랜드 처리를 위한 데이터 분류

In [4]:
data["index"] = range(len(data))
data = data[["index","방송일시","노출(분)","마더코드","상품코드","상품명","상품군","판매단가","취급액"]]

In [5]:
cloths = copy.deepcopy(data[data["상품군"]=="의류"][["index","상품명"]])
inwear = copy.deepcopy(data[data["상품군"]=="속옷"][["index","상품명"]])
kitchen = copy.deepcopy(data[data["상품군"]=="주방"][["index","상품명"]])
food = copy.deepcopy(data[data["상품군"]=="농수축"][["index","상품명"]])
skins = copy.deepcopy(data[data["상품군"]=="이미용"][["index","상품명"]])
digital = copy.deepcopy(data[data["상품군"]=="가전"][["index","상품명"]])
daily = copy.deepcopy(data[data["상품군"]=="생활용품"][["index","상품명"]])
health = copy.deepcopy(data[data["상품군"]=="건강기능"][["index","상품명"]])
merchandise = copy.deepcopy(data[data["상품군"]=="잡화"][["index","상품명"]])
nonp = copy.deepcopy(data[data["상품군"]=="무형"][["index","상품명"]])
furniture = copy.deepcopy(data[data["상품군"]=="가구"][["index","상품명"]])
bedding = copy.deepcopy(data[data["상품군"]=="침구"][["index","상품명"]])

cloths_prd = list(map(str.split,cloths["상품명"]))
inwear_prd = list(map(str.split,inwear["상품명"]))
kitchen_prd = list(map(str.split,kitchen["상품명"]))
food_prd = list(map(str.split,food["상품명"]))
skins_prd = list(map(str.split,skins["상품명"]))
digital_prd = list(map(str.split,digital["상품명"]))
daily_prd = list(map(str.split,daily["상품명"]))
health_prd = list(map(str.split,health["상품명"]))
merchandise_prd = list(map(str.split,merchandise["상품명"]))
nonp_prd = list(map(str.split,nonp["상품명"]))
furniture_prd = list(map(str.split,furniture["상품명"]))
bedding_prd = list(map(str.split,bedding["상품명"]))

In [6]:
cloths['브랜드'] = fn.fn_cloths_brand(cloths_prd)
inwear['브랜드'] = fn.fn_inwear_brand(inwear_prd)
kitchen['브랜드'] = fn.fn_kitchen_brand(kitchen_prd)
food['브랜드'] = fn.fn_food_brand(food_prd)
skins['브랜드'] = fn.fn_skins_brand(skins_prd)
digital['브랜드'] = fn.fn_digital_brand(digital_prd)
daily['브랜드'] = fn.fn_daily_brand(daily_prd)
health['브랜드'] = fn.fn_health_brand(health_prd)
merchandise['브랜드'] = fn.fn_merchandise_brand(merchandise_prd)
nonp['브랜드'] = fn.fn_nonp_brand(nonp_prd)
furniture['브랜드'] = fn.fn_furniture_brand(furniture_prd)
bedding['브랜드'] = fn.fn_bedding_brand(bedding_prd)

In [7]:
pd.unique(merchandise['브랜드'])

array(['에펨', '로베르타', '가이거', '세인트스코트', '도스문도스', '마스케라', '엘르', '엘유티',
       '칼리베이직', '아가타', '엘리자베스아덴', '삭루츠', '스프리스', '플로쥬', '제옥스', '레코바',
       '트레스패스', '안드레아바나리얼카이만', '썸덱스', 'DIOR', '프라다', '기라로쉬', '코치', '메이듀',
       '시스마르스', '오델로', 'AAD', '레노마', '루이띠에', '생쥴랑', '골드파일', '알비에로', '바치',
       '구찌', '에트로', '갈란테', 'RYN', '메디아글램', '에버라스트', '트리플에이', '아르테사노',
       '버버리', '마이클코어스'], dtype=object)

### 상품종류 처리

In [8]:
cloths['상품종류'] = fn.fn_prd_cloths(cloths)
inwear['상품종류'] = fn.fn_prd_inwear(inwear)
kitchen['상품종류'] = fn.fn_prd_kitchen(kitchen)
food['상품종류'] = fn.fn_prd_food(food)
skins['상품종류'] = fn.fn_prd_skins(skins)
digital['상품종류'] = fn.fn_prd_digital(digital)
daily['상품종류'] = fn.fn_prd_daily(daily)
health['상품종류'] = fn.fn_prd_health(health)
merchandise['상품종류'] = fn.fn_prd_merchandise(merchandise)
nonp['상품종류'] = fn.fn_prd_nonp(nonp)
furniture['상품종류'] = fn.fn_prd_furniture(furniture)
bedding['상품종류'] = fn.fn_prd_bedding(bedding)

In [9]:
brands_df = pd.concat([bedding,cloths,daily,digital,food,furniture,health,inwear,kitchen,merchandise,nonp,skins],axis=0)
data = pd.merge(data,brands_df[['index','브랜드','상품종류']],on='index')

del data['index']

In [10]:
data.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', '브랜드',
       '상품종류'],
      dtype='object')

In [11]:
data.isnull().sum()

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액        937
브랜드       2196
상품종류         0
dtype: int64

### 브랜드이름이 없는경우 Nobrand라고 대체하기

In [12]:
data["브랜드"].replace(np.nan,"Nobrand",inplace=True)

### 모든 행에 대해 브랜드명과 상품명을 합친 후 크롤링을 위해 list로 만들기

In [13]:
anw=[]
for i in range(len(data)):
    anw.append([])

for i in range(len(data)):
    for j in range(len(data['상품종류'][i])):
        if data['브랜드'][i] == 'Nobrand':
            a  = data['상품종류'][i][j]
            anw[i].append(a)
        else:
            a = data['브랜드'][i]+data['상품종류'][i][j]
            anw[i].append(a)
anw

[['테이트니트'],
 ['테이트니트'],
 ['테이트니트'],
 ['테이트니트'],
 ['테이트니트'],
 ['테이트니트'],
 ['오모떼브라', '오모떼레이스'],
 ['오모떼브라', '오모떼레이스'],
 ['오모떼브라', '오모떼레이스'],
 ['CERINI팬츠', 'CERINI기모'],
 ['CERINI팬츠', 'CERINI기모'],
 ['CERINI팬츠', 'CERINI기모'],
 ['보코무스탕', '보코리버시블'],
 ['보코무스탕', '보코리버시블'],
 ['보코무스탕', '보코리버시블'],
 ['CERINI니트', 'CERINI기모'],
 ['CERINI니트', 'CERINI기모'],
 ['CERINI니트', 'CERINI기모'],
 ['크로커다일브라', '크로커다일팬티', '크로커다일브라팬티'],
 ['크로커다일브라', '크로커다일팬티', '크로커다일브라팬티'],
 ['에코라믹냄비'],
 ['에코라믹냄비'],
 ['에코라믹냄비'],
 ['오모떼레이스'],
 ['오모떼레이스'],
 ['그렉노먼코트', '그렉노먼구스', '그렉노먼벤치코트'],
 ['그렉노먼코트', '그렉노먼구스', '그렉노먼벤치코트'],
 ['그렉노먼코트', '그렉노먼구스', '그렉노먼벤치코트'],
 ['그렉노먼코트', '그렉노먼구스', '그렉노먼벤치코트'],
 ['뽕셰프갈비탕'],
 ['뽕셰프갈비탕'],
 ['뽕셰프갈비탕'],
 ['갑오징어', '손질갑오징어'],
 ['갑오징어', '손질갑오징어'],
 ['농협쌀', '농협햅쌀'],
 ['비버리힐스폴로클럽기초세트'],
 ['비버리힐스폴로클럽기초세트'],
 ['비버리힐스폴로클럽기초세트'],
 ['우리바다손질왕꼬막'],
 ['우리바다손질왕꼬막'],
 ['우리바다손질왕꼬막'],
 ['푸마브라', '푸마브라세트'],
 ['푸마브라', '푸마브라세트'],
 ['푸마브라', '푸마브라세트'],
 ['LG세탁기'],
 ['LG세탁기'],
 ['LG세탁기'],
 ['LG세탁기'],
 ['LG세탁기'],
 ['LG세탁기'],
 ['마리노블코트',

### 브랜드+상품 변수 추가

In [14]:
data['브랜드상품'] = anw

### 수량변수 추가 및 결측치 제거

In [15]:
data["수량"] = data["취급액"] / data["판매단가"]
data["취급액"].fillna(0,inplace=True)
data["수량"].fillna(0,inplace=True)
data["수량"]= list(map(math.ceil,data["수량"]))
data["노출(분)"].fillna(method='ffill',inplace=True)
data["취급액"].replace(50000,0,inplace=True)
data["방송일시"] = pd.to_datetime(data["방송일시"])

In [16]:
data

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,상품종류,브랜드상품,수량
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트,[니트],[테이트니트],53
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트,[니트],[테이트니트],110
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트,[니트],[테이트니트],82
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트,[니트],[테이트니트],175
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트,[니트],[테이트니트],168
...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,0.0,삼성화재,"[보험, 주택화재보험]","[삼성화재보험, 삼성화재주택화재보험]",0
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,0.0,삼성화재,"[보험, 주택화재보험]","[삼성화재보험, 삼성화재주택화재보험]",0
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,0.0,삼성화재,"[보험, 주택화재보험]","[삼성화재보험, 삼성화재주택화재보험]",0
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,0.0,더케이,[상조서비스],[더케이상조서비스],0


### 방송_월,방송_일 변수 추가

In [17]:
import datetime
dd=[]
for i in range(len(data)):
    dd.append(datetime.datetime.strptime(data["방송일시"][i].strftime("%Y-%m-%d"),"%Y-%m-%d").date())
data["방송날짜"] = dd

month=[]
for i in range(len(data)):
    month.append(data["방송일시"][i].month)
data["방송_월"] = month

day=[]
for i in range(len(data)):
    day.append(data["방송일시"][i].day)
data["방송_일"] = day

In [18]:
data.isna().sum()

방송일시     0
노출(분)    0
마더코드     0
상품코드     0
상품명      0
상품군      0
판매단가     0
취급액      0
브랜드      0
상품종류     0
브랜드상품    0
수량       0
방송날짜     0
방송_월     0
방송_일     0
dtype: int64

### 상품군별 최대 일수 병합

In [19]:
product =[]
days = []
for i in np.unique(data["상품군"]):
    diff=[]
    code = np.unique(data[data["상품군"]==i]["상품코드"])
    for j in range(0,len(code)):
        a=pd.DataFrame(data[data["상품코드"]==code[j]])
        b = max(a["방송날짜"]) -  min(a["방송날짜"])
        diff.append(b.days)
        Max_diff = max(diff)
    product.append(i)
    days.append(Max_diff)


Duration = pd.DataFrame({"상품군" : product, "판매일수" : days})

In [20]:
data = pd.merge(data,Duration)

In [21]:
data.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', '브랜드',
       '상품종류', '브랜드상품', '수량', '방송날짜', '방송_월', '방송_일', '판매일수'],
      dtype='object')

In [22]:
data.to_json('data_v1_COM.json',orient='table')